<a href="https://colab.research.google.com/github/NandakumarGunalan/MLLaboratory/blob/main/Copy_of_lec_6a_imbalance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Credit Fraud & Credit Default
## Dealing with Imbalanced Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Imports

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split

Get the data at https://www.kaggle.com/mlg-ulb/creditcardfraud

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/MLLab/creditcard_fraud.csv')
data.shape

(284807, 31)

Target: 1 = fraud;
0 = ok

Each row = 1 transaction

Amount = amount of transaction

V1-28 = masked features for privacy... not ideal (see credit default dataset for named features)

Time = seconds since first transaction

In [ ]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
# Proportion of positive samples
data.Class.mean()

np.float64(0.001727485630620034)

## Train and validation split

In [ ]:
X = data.iloc[:,:-1]
y = data.Class.values

In [ ]:
def train_test_split_scaled(x: pd.DataFrame,
                            y: np.array,
                            test_size: float = 0.2,
                            random_state: int = 8675309,
                            stratify: np.array = None,
                            sample: float = None) -> tuple:
  """Given an input feature matrix and target vector, split into train/test
  split then scale the features.

  Arugments:
    x (pd.DataFrame): The matrix of features.
    y (np.array): A vector of targets.

  Returns:
    A tuple containing x_train, x_val, y_train and y_val.
  """

  if sample is not None and sample < 1 and sample > 0:
    _, x, _, y = train_test_split(
        x, y, test_size=sample, random_state=random_state, stratify=stratify)
    if stratify is not None:
      stratify=y

  x_train, x_val, y_train, y_val = train_test_split(
      x, y, test_size=test_size, random_state=random_state, stratify=stratify)
  scaler = StandardScaler()
  x_train = scaler.fit_transform(x_train)
  x_val = scaler.transform(x_val)
  return x_train, x_val, y_train, y_val

## Logistic regression baseline

In [ ]:
x_train, x_val, y_train, y_val = train_test_split_scaled(
    X, y, stratify=y, sample=0.1)

In [ ]:
y_train.shape, y_val.shape

((22784,), (5697,))

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

y_hat_val = lr.predict_proba(x_val)[:, 1]
precision, recall, _ = precision_recall_curve(y_val, y_hat_val)
aucpr = auc(recall, precision)
aucroc = roc_auc_score(y_val, y_hat_val)
print('No sampling: aucpr = %.3f; aucroc = %.3f' % (aucpr, aucroc))

No sampling: aucpr = 0.613; aucroc = 0.937


## Logistic regression with weights

#Performing class-weight tuning for a logistic regression model on an imbalanced binary classification problem.

In datasets where one class appears much less frequently than the other, the model often becomes biased toward predicting the majority class. To address this, different weights are assigned to class 1 (the minority class) to make the model pay more attention to it during training.

*  loops over several possible weight values. such as 2, 5, 10, 100, 500, 1000, and 10000

* for each weight, it builds a logistic regression model using that class-weight configuration.


* The logistic regression is trained using the training dataset, and then its performance is evaluated on a validation dataset by computing prediction probabilities for class 1.


* Using these probabilities, the code calculates a precision–recall curve and computes the area under that curve (AUC-PR), which is a strong metric for imbalanced datasets.


* It also computes the ROC AUC score for additional performance insight. Finally, the code prints the weight value alongside its AUC-PR and AUC-ROC scores to help identify which class weight provides the best balance between detecting the minority class and maintaining overall model performance.

In [ ]:
for w in [2, 5, 10, 100, 500, 1000, 10000]:
  weights = {0:1.0, 1:w}
  lr = LogisticRegression(solver='lbfgs', max_iter = 1000, class_weight=weights)
  lr.fit(x_train, y_train)
  y_hat_val = lr.predict_proba(x_val)[:, 1]
  precision, recall, thresholds = precision_recall_curve(y_val, y_hat_val)
  auc_precision_recall = auc(recall, precision)
  auc_roc = roc_auc_score(y_val, y_hat_val)
  print('weight: %d auc precision recall %.3f auc roc %.3f' %
        (w, auc_precision_recall, auc_roc))

weight: 2 auc precision recall 0.638 auc roc 0.950
weight: 5 auc precision recall 0.640 auc roc 0.952
weight: 10 auc precision recall 0.617 auc roc 0.934
weight: 100 auc precision recall 0.487 auc roc 0.891
weight: 500 auc precision recall 0.562 auc roc 0.882
weight: 1000 auc precision recall 0.556 auc roc 0.882
weight: 10000 auc precision recall 0.618 auc roc 0.927


## Undersampling

`pip install imblearn # in case it's not preinstalled`

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import CondensedNearestNeighbour, TomekLinks
from collections import Counter

In [ ]:
# @title split data again using a 10% stratified sample
x_train, x_val, y_train, y_val = train_test_split_scaled(
    X, y, stratify=y, sample=0.1)

## performs undersampling using the Condensed Nearest Neighbor (CNN) algorithm

 CNN algorithm handle class imbalance in a training dataset.

 Before undersampling, the code prints the original distribution of classes using Counter(y_train) so you can see how imbalanced the dataset is.


 `The CNN method is a type of prototype selection that removes redundant majority-class samples while trying to keep only the most informative instances.`

It works by iteratively selecting only the points from the majority class that are necessary to correctly classify a subset of the data using a nearest-neighbor classifier.

This means the algorithm retains minority-class samples but removes many majority-class samples that do not contribute significantly to decision boundaries.


We initializes the CNN undersampler with a fixed random seed to ensure reproducibility and with one nearest neighbor as the classification rule.


 It then applies fit_resample to create a new, reduced training dataset (x_train_cnn, y_train_cnn). T



 The `%time magic` command measures how long this resampling takes.


 Finally, print the class distribution of the resampled dataset, allowing you to compare before and after undersampling and observe how CNN reduces the dataset while keeping essential structure for classification.

In [ ]:
# @title undersampling with Condensed Nearest Neighbor
print('Original dataset shape %s' % Counter(y_train))

cnn = CondensedNearestNeighbour(random_state=42, n_neighbors=1)
%time x_train_cnn, y_train_cnn = cnn.fit_resample(x_train, y_train)

print('CNN: Resampled dataset shape %s' % Counter(y_train_cnn))

Original dataset shape Counter({np.int64(0): 22745, np.int64(1): 39})
CPU times: user 2.37 s, sys: 3.22 ms, total: 2.38 s
Wall time: 2.39 s
CNN: Resampled dataset shape Counter({np.int64(0): 101, np.int64(1): 39})


In [ ]:
lr = LogisticRegression()
lr.fit(x_train_cnn, y_train_cnn)

y_hat_val = lr.predict_proba(x_val)[:, 1]
precision, recall, _ = precision_recall_curve(y_val, y_hat_val)
aucpr = auc(recall, precision)
aucroc = roc_auc_score(y_val, y_hat_val)
print('CNN: aucpr = %.3f; aucroc = %.3f' % (aucpr, aucroc))

CNN: aucpr = 0.130; aucroc = 0.707


## performs undersampling using the Tomek Links method to clean the training dataset.

Tomek Links are pairs of samples from opposite classes that are each other’s nearest neighbors and lie very close to the class boundary.

These pairs often indicate overlapping or noisy data. The Tomek Links algorithm removes the majority-class examples in these pairs to make the boundary cleaner and reduce class imbalance. The code first prints the original class distribution, applies Tomek Links with fit_resample to remove those borderline majority samples, and finally prints the new class distribution so you can see how the dataset changed.

In [ ]:
# @title undersampling with TomekLinks
print('Original dataset shape %s' % Counter(y_train))

tl = TomekLinks()
%time x_train_tl, y_train_tl = tl.fit_resample(x_train, y_train)

print('TomekLinks: Resampled dataset shape %s' % Counter(y_train_tl))

Original dataset shape Counter({np.int64(0): 22745, np.int64(1): 39})
CPU times: user 3.2 s, sys: 5.28 ms, total: 3.21 s
Wall time: 3.21 s
TomekLinks: Resampled dataset shape Counter({np.int64(0): 22744, np.int64(1): 39})


In [ ]:
lr = LogisticRegression()
lr.fit(x_train_tl, y_train_tl)

y_hat_val = lr.predict_proba(x_val)[:, 1]
precision, recall, _ = precision_recall_curve(y_val, y_hat_val)
aucpr = auc(recall, precision)
aucroc = roc_auc_score(y_val, y_hat_val)
print('TomekLinks: aucpr = %.3f; aucroc = %.3f' % (aucpr, aucroc))

TomekLinks: aucpr = 0.702; aucroc = 0.971


## Oversampling

##Oversampling using SMOTE (Synthetic Minority Oversampling Technique)


to increase the number of minority-class samples. Instead of simply duplicating minority samples, SMOTE creates new synthetic examples by interpolating between existing minority samples and their nearest neighbors. This helps balance the dataset and reduces overfitting. The code starts by printing the original class distribution, applies SMOTE with a sampling strategy of 0.5 (meaning the minority class will become half the size of the majority class), and then prints the new class distribution after generating synthetic minority samples.

In [ ]:
# @title oversampling with SMOTE
print('Original dataset shape %s' % Counter(y_train))

smote = SMOTE(sampling_strategy=0.5)
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)

print('SMOTE: Resampled dataset shape %s' % Counter(y_train_smote))

Original dataset shape Counter({np.int64(0): 22745, np.int64(1): 39})
SMOTE: Resampled dataset shape Counter({np.int64(0): 22745, np.int64(1): 11372})


In [ ]:
# Default solver l-bfgs-b failed to converge.
lr = LogisticRegression(solver='newton-cg')
lr.fit(x_train_smote, y_train_smote)

y_hat_val = lr.predict_proba(x_val)[:, 1]
precision, recall, _ = precision_recall_curve(y_val, y_hat_val)
aucpr = auc(recall, precision)
aucroc = roc_auc_score(y_val, y_hat_val)
print('SMOTE: aupr = %.3f; auroc = %.3f' % (aucpr, aucroc))

SMOTE: aupr = 0.498; auroc = 0.893


In [ ]:
from imblearn.under_sampling import OneSidedSelection, RandomUnderSampler

In [ ]:
x_train, x_val, y_train, y_val = train_test_split_scaled(X, y, stratify=y, sample=0.1)

In [ ]:
x_train.shape

(22784, 30)

## random undersampling using the RandomUnderSampler class.


Unlike more sophisticated methods, random undersampling simply removes random samples from the majority class to reduce imbalance. The sampling_strategy=0.02 parameter means the minority class will become 2% of the majority class after resampling. First, the code prints the original class distribution, then performs the undersampling with fit_resample, and finally prints the new class distribution so you can see how many samples were removed from the majority class.

In [ ]:
# @title undersample with RandomUnderSampler
print('Original dataset shape %s' % Counter(y_train))

rus = RandomUnderSampler(sampling_strategy=0.02)
x_train_rus, y_train_rus = rus.fit_resample(x_train, y_train)

print('RandomUnderSampler: Original dataset shape %s' % Counter(y_train_rus))

Original dataset shape Counter({np.int64(0): 22745, np.int64(1): 39})
RandomUnderSampler: Original dataset shape Counter({np.int64(0): 1950, np.int64(1): 39})


In [ ]:
lr = LogisticRegression()
lr.fit(x_train_rus, y_train_rus)
y_hat_val = lr.predict_proba(x_val)[:, 1]
precision, recall, _ = precision_recall_curve(y_val, y_hat_val)
aucpr = auc(recall, precision)
aucroc = roc_auc_score(y_val, y_hat_val)
print('RandomUnderSampler: aucpr %.3f; aucroc %.3f' % (aucpr, aucroc))

RandomUnderSampler: aucpr 0.710; aucroc 0.909


## performs undersampling using the One-Sided Selection (OSS) method


which removes noisy and redundant majority-class examples. OSS works by combining Tomek Links removal and a nearest-neighbor classifier to keep only the most informative majority samples. The goal is to clean the decision boundary and reduce class imbalance without discarding useful data. The code first prints the original class distribution, applies One-Sided Selection with specific parameters (n_neighbors=1, n_seeds_S=10), and then prints the new class distribution for the resampled dataset.

In [ ]:
# @title undersample with OneSidedSelection
print('Original dataset shape %s' % Counter(y_train))

oss = OneSidedSelection(n_neighbors=1, n_seeds_S=10, n_jobs=-1)
x_train_oss, y_train_oss = oss.fit_resample(x_train, y_train)

print('OneSidedSelection: Original dataset shape %s' % Counter(y_train_rus))

Original dataset shape Counter({np.int64(0): 22745, np.int64(1): 39})
OneSidedSelection: Original dataset shape Counter({np.int64(0): 1950, np.int64(1): 39})


In [ ]:
lr = LogisticRegression()
lr.fit(x_train_oss, y_train_oss)
y_hat_val = lr.predict_proba(x_val)[:, 1]
precision, recall, _ = precision_recall_curve(y_val, y_hat_val)
aucpr = auc(recall, precision)
aucroc = roc_auc_score(y_val, y_hat_val)
print('OneSidedSelection: aucpr %.3f; aucroc %.3f' % (aucpr, aucroc))

OneSidedSelection: aucpr 0.696; aucroc 0.924


The smote_cv function performs cross-validation with SMOTE oversampling to evaluate a Logistic Regression model on an imbalanced dataset. It uses Stratified K-Fold to split the data into training and validation sets, maintaining the original class proportions. For each fold, the features are standardized using the training data, and the SMOTE algorithm generates synthetic minority-class samples to balance the training set. A Logistic Regression model is then trained, and predictions are made on the validation set. After all folds are processed, the function concatenates all predictions and true labels, computes the precision-recall curve, and calculates two evaluation metrics: AUC-PR (which is more sensitive to class imbalance) and AUC-ROC. These metrics are returned to summarize the model’s overall performance.

In [ ]:
def stratified_sample(x: pd.DataFrame,
                      y: np.array,
                      sample: float = 0.1,
                      random_state: int = 8675309) -> tuple:
  _, x, _, y = train_test_split(
      x, y, test_size=sample, random_state=random_state, stratify=y)
  return x, y

X = data.iloc[:,:-1]
y = data.Class.values

x_sample, y_sample = stratified_sample(X, y)

x_train, x_val, y_train, y_val = train_test_split(
    x_sample, y_sample, test_size=0.2, random_state=42, stratify=y_sample)

In [ ]:
x_train.shape

(22784, 30)

In [ ]:
def smote_cv(x, y, folds: int = 3, sampling_strategy: float = 0.5) -> tuple:
  predicted = []
  actual = []

  kf = StratifiedKFold(n_splits = folds)
  kf.get_n_splits(x, y)

  for i, (train_index, val_index) in enumerate(kf.split(x, y)):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Scale using training data.
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.transform(x_val)

    # Oversample with SMOTE.
    smote = SMOTE(sampling_strategy=sampling_strategy, random_state=i)
    x_train, y_train = smote.fit_resample(x_train, y_train)

    # Train LR.
    lr = LogisticRegression(max_iter = 1000).fit(x_train, y_train)

    # Predict on validation set.
    predicted.append(lr.predict_proba(x_val)[:, 1])
    actual.append(y_val)

  actual = np.concatenate(actual)
  predicted = np.concatenate(predicted)
  precision, recall, _ = precision_recall_curve(actual, predicted)

  aucpr = auc(recall, precision)
  aucroc = roc_auc_score(actual, predicted)

  return aucpr, aucroc

In [ ]:
x_train.shape, y_train.shape

((22784, 30), (22784,))

In [ ]:
from sklearn.model_selection import StratifiedKFold

for s in [0.01, 0.05, 0.1, 0.2, 0.5]:
  aucpr, aucroc = smote_cv(
      x_train.values, y_train, folds=3, sampling_strategy=s)
  print("sampling_strategy=%.3f; aucpr=%.3f; aucroc=%.3f" % (s, aucpr, aucroc))

sampling_strategy=0.010; aucpr=0.738; aucroc=0.947
sampling_strategy=0.050; aucpr=0.694; aucroc=0.942
sampling_strategy=0.100; aucpr=0.685; aucroc=0.936
sampling_strategy=0.200; aucpr=0.649; aucroc=0.934
sampling_strategy=0.500; aucpr=0.626; aucroc=0.933


## References
1. https://www.kaggle.com/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets
2. https://machinelearningmastery.com/undersampling-algorithms-for-imbalanced-classification/